In [1]:
import numpy as np
import json
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text # See https://github.com/tensorflow/hub/issues/463

In [2]:
train_file = open("conala-corpus-v1.1/conala-corpus/conala-train.json", "r")
train_json = json.load(train_file)

print(len(train_json))
print(train_json[0])

2379
{'intent': 'How to convert a list of multiple integers into a single integer?', 'rewritten_intent': "Concatenate elements of a list 'x' of multiple integers to a single integer", 'snippet': 'sum(d * 10 ** i for i, d in enumerate(x[::-1]))', 'question_id': 41067960}


In [3]:
base_sentences = [item["intent"] for item in train_json]
codes = [item["snippet"] for item in train_json]

print(base_sentences[0])
print(codes[0])

How to convert a list of multiple integers into a single integer?
sum(d * 10 ** i for i, d in enumerate(x[::-1]))


In [4]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1", trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 128].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 128].



sample_decoder = Decoder(num_layers=2, d_model=512, num_heads=8,
                         dff=2048, target_vocab_size=8000,
                         maximum_position_encoding=5000)
temp_input = tf.random.uniform((64, 26), dtype=tf.int64, minval=0, maxval=200)

output, attn = sample_decoder(temp_input,
                              enc_output=sample_encoder_output,
                              training=False,
                              look_ahead_mask=None,
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape